In [1]:
import pandas as pd
import numpy as np
import dgl.nn as dglnn
from dgl import from_networkx
import torch.nn as nn
import torch as th
import torch.nn.functional as F
import dgl.function as fn
from dgl.data.utils import load_graphs
import networkx as nx
import pandas as pd
import socket
import struct
import random
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [2]:
import networkx
import torch
import numpy as np
import pandas as pd
from sklearn.metrics import *
from torch_geometric.loader import NeighborSampler, NeighborLoader
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GATConv, ResGatedGraphConv, GATv2Conv, SAGEConv, GENConv, DeepGCNLayer, PairNorm, GINConv
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import torch.nn.functional as F
from imblearn.under_sampling import RandomUnderSampler
pd.options.mode.use_inf_as_na = True
from collections import Counter
from sklearn.feature_selection import SelectFromModel
import torch.nn as nn
import time
import pickle
from torch.nn import LayerNorm, Linear, ReLU
from torch_scatter import scatter
from tqdm import tqdm
from torch_geometric.loader import RandomNodeSampler
import math
import copy
from sklearn.metrics import f1_score
from torch.optim import lr_scheduler
from sklearn.manifold import TSNE

In [3]:
class GATLayer(nn.Module):
    def __init__(self, in_dim, out_dim, e_dim):
        super(GATLayer, self).__init__()
        self.w1 = nn.Linear(in_dim + e_dim, out_dim, bias=False)
        self.w2 = nn.Linear(in_dim + out_dim, out_dim, bias=False)
        self.w_att = nn.Linear(in_dim + out_dim, 1, bias=False)
        self.reset_parameters()
    def reset_parameters(self):
        gain = nn.init.calculate_gain('relu')
        nn.init.xavier_normal_(self.w1.weight, gain=gain)
        nn.init.xavier_normal_(self.w2.weight, gain=gain)
        nn.init.xavier_normal_(self.w_att.weight, gain=gain)
    def edge_attention(self, edges):
        z = torch.cat([edges.src['h'], edges.data['m']], -1)
        a = self.w_att(z)
        alpha = F.leaky_relu(a)
        return {'e': alpha}
    def msg1(self, edges):
        return {'m': self.w1(th.cat([edges.src['h'], edges.data['h']], -1))}
    
    def message_func(self, edges):
        return {'z': edges.data['m'], 'e': edges.data['e']}
    def reduce_func(self, nodes):
        alpha = F.softmax(nodes.mailbox['e'], dim=1) # 归一化每一条入边的注意力系数
        h = torch.sum(alpha * nodes.mailbox['z'], dim=1)
        return {'h_neigh': h}
    def forward(self, g_dgl, hfeat, efeat):
        with g_dgl.local_scope():
            g = g_dgl
            g.ndata['h'] = hfeat
            g.edata['h'] = efeat
            g.apply_edges(self.msg1)
            g.apply_edges(self.edge_attention) # 为每一条边获得其注意力系数
            g.update_all(self.message_func, self.reduce_func)
            g.ndata['h'] = F.relu(self.w2(th.cat([g.ndata['h'], g.ndata['h_neigh']], -1)))
            return g.ndata['h']
class Model(nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim, e_dim):
        super().__init__()
        self.layer1 = GATLayer(in_dim, hidden_dim, e_dim)
        self.layer2 = GATLayer(hidden_dim, out_dim, e_dim)
        self.pred = MLPPredictor(out_dim, 10)
    def forward(self, g, nfeats, efeats):
        nfeats = self.layer1(g, nfeats, efeats)
        nfeats = self.layer2(g, nfeats, efeats)
        return self.pred(g, nfeats)
class MLPPredictor(nn.Module):
    def __init__(self, in_features, out_classes):
        super().__init__()
        self.W = nn.Linear(in_features * 2, out_classes)
    def apply_edges(self, edges):
        h_u = edges.src['h']
        h_v = edges.dst['h']
        score = self.W(th.cat([h_u, h_v], 1))
        return {'score': score}
    def forward(self, graph, h):
        with graph.local_scope():
            graph.ndata['h'] = h
            graph.apply_edges(self.apply_edges)
            return graph.edata['score']

In [4]:
class SAGELayer(nn.Module):
    def __init__(self, ndim_in, edims, ndim_out, activation):
        super(SAGELayer, self).__init__()
        ### force to outut fix dimensions
        self.W_msg = nn.Linear(ndim_in + edims, ndim_out)
        ### apply weight
        self.W_apply = nn.Linear(ndim_in + ndim_out, ndim_out)
        self.activation = activation
        self.embedector = torch.nn.Embedding(435887, 78)

    def message_func(self, edges):
        return {'m': self.W_msg(th.cat([edges.src['h'], edges.data['h']], 2))}

    def forward(self, g_dgl, nfeats, efeats):
        with g_dgl.local_scope():
            g = g_dgl
            g.ndata['h'] = nfeats
            g.edata['h'] = efeats
            # Eq4
            g.update_all(self.message_func, fn.mean('m', 'h_neigh'))
            # Eq5          
            g.ndata['h'] = F.relu(self.W_apply(th.cat([g.ndata['h'], g.ndata['h_neigh']], 2)))
            return g.ndata['h']


class SAGE(nn.Module):
    def __init__(self, ndim_in, ndim_out, edim, activation, dropout):
        super(SAGE, self).__init__()
        self.layers = nn.ModuleList()
        self.layers.append(SAGELayer(ndim_in, edim, 128, activation))
        self.layers.append(SAGELayer(128, edim, ndim_out, activation))
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, g, nfeats, efeats):
        for i, layer in enumerate(self.layers):
            if i != 0:
                nfeats = self.dropout(nfeats)
            nfeats = layer(g, nfeats, efeats)
        return nfeats.sum(1)

In [5]:
G = load_graphs("./mydata/unswnb15_train_data2.bin")[0][0]

In [6]:
G

Graph(num_nodes=129739, num_edges=358047,
      ndata_schemes={}
      edata_schemes={'label': Scheme(shape=(), dtype=torch.int64), 'h': Scheme(shape=(40,), dtype=torch.float32)})

In [7]:
G.nodes()

tensor([     0,      1,      2,  ..., 129736, 129737, 129738])

In [4]:
G.ndata['h'] = th.ones(G.num_nodes(), G.edata['h'].shape[1]) 
G.edata['train_mask'] = th.ones(len(G.edata['h']), dtype = th.bool)

NameError: name 'G' is not defined

In [6]:
G

Graph(num_nodes=406258, num_edges=1149379,
      ndata_schemes={'h': Scheme(shape=(40,), dtype=torch.float32)}
      edata_schemes={'label': Scheme(shape=(), dtype=torch.int64), 'h': Scheme(shape=(40,), dtype=torch.float32), 'train_mask': Scheme(shape=(), dtype=torch.bool)})

In [7]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(G.edata['label'].cpu().numpy()),
                                                 G.edata['label'].cpu().numpy())
class_weights = th.FloatTensor(class_weights)

/home/xiaoyujie/anaconda3/envs/densegat2/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass classes=[0 1 2 3 4 5 6 7 8 9], y=[0 0 0 ... 0 0 0] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


In [8]:
device = 'cuda:3'

In [9]:
G = G.to(device)

In [10]:
class_weights = class_weights.to(device)

In [11]:
node_features = G.ndata['h']
edge_features = G.edata['h']

edge_label = G.edata['label']
train_mask = G.edata['train_mask']

In [12]:
node_features.size()

torch.Size([406258, 40])

In [13]:
in_dim = node_features.size(1)
hidden_dim = 20
out_dim = 10
e_dim = in_dim

In [14]:
model = Model(in_dim, hidden_dim, out_dim, e_dim)

In [15]:
opt = th.optim.Adam(model.parameters())

In [16]:
criterion = nn.CrossEntropyLoss()

In [17]:
import time

In [18]:
def compute_accuracy(pred, labels):
    return (pred.argmax(1) == labels).float().mean().item()

In [20]:
weight = torch.load('egat_unswnb15_model' + str(14900))

In [21]:
model.load_state_dict(weight)

<All keys matched successfully>

In [41]:
model = model.to(device)
for epoch in range(1 ,15000):
    pred = model(G, node_features, edge_features)
    loss = criterion(pred[train_mask], edge_label[train_mask])
    opt.zero_grad()
    loss.backward()
    opt.step()
    print('loss',loss.item(),'Epoch:', epoch ,' Training acc:', compute_accuracy(pred[train_mask], edge_label[train_mask]))
    if epoch % 100 == 0:
        torch.save(model.state_dict(), 'egat_unswnb15_model' + str(epoch))

loss 0.12728941440582275 Epoch: 1  Training acc: 0.9509613513946533
loss 0.1273033171892166 Epoch: 2  Training acc: 0.9506411552429199
loss 0.12730267643928528 Epoch: 3  Training acc: 0.9508882761001587
loss 0.12731419503688812 Epoch: 4  Training acc: 0.9506481289863586
loss 0.1273181289434433 Epoch: 5  Training acc: 0.9509230256080627
loss 0.12733949720859528 Epoch: 6  Training acc: 0.9509787559509277
loss 0.12734583020210266 Epoch: 7  Training acc: 0.9505480527877808
loss 0.12734884023666382 Epoch: 8  Training acc: 0.9509891867637634
loss 0.12734565138816833 Epoch: 9  Training acc: 0.9505001902580261
loss 0.12739503383636475 Epoch: 10  Training acc: 0.9509230256080627
loss 0.12739382684230804 Epoch: 11  Training acc: 0.9502400755882263
loss 0.1273752748966217 Epoch: 12  Training acc: 0.9509561061859131
loss 0.12736351788043976 Epoch: 13  Training acc: 0.9506446123123169
loss 0.12737911939620972 Epoch: 14  Training acc: 0.9508969783782959
loss 0.12736204266548157 Epoch: 15  Training a

loss 0.1272467076778412 Epoch: 121  Training acc: 0.9510030746459961
loss 0.12723949551582336 Epoch: 122  Training acc: 0.9509491324424744
loss 0.12724502384662628 Epoch: 123  Training acc: 0.9506829380989075
loss 0.1272590011358261 Epoch: 124  Training acc: 0.9509056210517883
loss 0.12727366387844086 Epoch: 125  Training acc: 0.9507229328155518
loss 0.12726865708827972 Epoch: 126  Training acc: 0.9508203864097595
loss 0.12726418673992157 Epoch: 127  Training acc: 0.951065719127655
loss 0.12728199362754822 Epoch: 128  Training acc: 0.9505175948143005
loss 0.127256378531456 Epoch: 129  Training acc: 0.9510692358016968
loss 0.12726198136806488 Epoch: 130  Training acc: 0.9509335160255432
loss 0.12725910544395447 Epoch: 131  Training acc: 0.9507107734680176
loss 0.1272473782300949 Epoch: 132  Training acc: 0.9508830308914185
loss 0.127253457903862 Epoch: 133  Training acc: 0.9506620168685913
loss 0.12724551558494568 Epoch: 134  Training acc: 0.950869083404541
loss 0.12724795937538147 Epoc

KeyboardInterrupt: 

In [39]:
model = model.to('cpu')
cm, cr = test(G_test, model)

In [40]:
print(cr)

              precision    recall  f1-score   support

           0     0.9928    0.9878    0.9903    299832
           1     0.5859    0.8352    0.6887     26716
           2     0.9179    0.6978    0.7929      8392
           3     0.3855    0.1492    0.2151      9812
           4     0.9918    0.9789    0.9853    129288
           5     0.5725    0.8190    0.6739       906
           6     0.6939    0.7990    0.7427     14548
           7     0.0000    0.0000    0.0000       104
           8     0.3667    0.0157    0.0302      1398
           9     0.0000    0.0000    0.0000      1606

    accuracy                         0.9435    492602
   macro avg     0.5507    0.5283    0.5119    492602
weighted avg     0.9423    0.9435    0.9398    492602



In [34]:
print(cr)

              precision    recall  f1-score   support

           0     0.9929    0.9880    0.9904    299832
           1     0.5889    0.8215    0.6860     26716
           2     0.8954    0.6995    0.7854      8392
           3     0.3807    0.2532    0.3041      9812
           4     0.9929    0.9789    0.9858    129288
           5     0.5664    0.8102    0.6667       906
           6     0.7438    0.7576    0.7507     14548
           7     0.0000    0.0000    0.0000       104
           8     0.4815    0.0186    0.0358      1398
           9     0.0079    0.0012    0.0022      1606

    accuracy                         0.9437    492602
   macro avg     0.5650    0.5329    0.5207    492602
weighted avg     0.9441    0.9437    0.9417    492602



In [28]:
print(cr)

              precision    recall  f1-score   support

           0     0.9922    0.9882    0.9902    299832
           1     0.5568    0.8932    0.6860     26716
           2     0.8696    0.6992    0.7752      8392
           3     0.3178    0.0222    0.0415      9812
           4     0.9938    0.9783    0.9860    129288
           5     0.5613    0.8190    0.6661       906
           6     0.7373    0.7612    0.7491     14548
           7     0.0000    0.0000    0.0000       104
           8     0.5000    0.0172    0.0332      1398
           9     0.0667    0.0012    0.0024      1606

    accuracy                         0.9431    492602
   macro avg     0.5595    0.5180    0.4930    492602
weighted avg     0.9405    0.9431    0.9362    492602



In [51]:
print(cr)

              precision    recall  f1-score   support

           0     0.9892    0.9879    0.9886    299832
           1     0.5592    0.8571    0.6768     26716
           2     0.9335    0.7064    0.8042      8392
           3     0.3923    0.0136    0.0262      9812
           4     0.9944    0.9774    0.9858    129288
           5     0.5639    0.7550    0.6456       906
           6     0.6808    0.8043    0.7374     14548
           7     0.0000    0.0000    0.0000       104
           8     0.0000    0.0000    0.0000      1398
           9     0.0909    0.0012    0.0025      1606

    accuracy                         0.9418    492602
   macro avg     0.5204    0.5103    0.4867    492602
weighted avg     0.9386    0.9418    0.9344    492602



In [48]:
print(cr)

              precision    recall  f1-score   support

           0     0.9964    0.9875    0.9919    299832
           1     0.5472    0.9163    0.6852     26716
           2     0.9653    0.6372    0.7676      8392
           3     0.8125    0.0026    0.0053      9812
           4     0.9938    0.9766    0.9851    129288
           5     0.5728    0.5210    0.5457       906
           6     0.6557    0.7771    0.7112     14548
           7     0.0000    0.0000    0.0000       104
           8     0.0000    0.0000    0.0000      1398
           9     0.3030    0.0062    0.0122      1606

    accuracy                         0.9419    492602
   macro avg     0.5847    0.4824    0.4704    492602
weighted avg     0.9510    0.9419    0.9347    492602



In [22]:
def test(G_test, model):
    test_node_features = G_test.ndata['h']
    test_edge_features = G_test.edata['h']
    y_true = G_test.edata['label'].detach().numpy()
    pred = model(G_test, test_node_features, test_edge_features)
    y_pred = pred.detach().numpy()
    y_pred = np.argmax(y_pred, -1)
    cm = confusion_matrix(y_true, y_pred)
    cr = classification_report(y_true, y_pred, digits=4)
    return cm, cr

In [23]:
G

Graph(num_nodes=406258, num_edges=1149379,
      ndata_schemes={'h': Scheme(shape=(40,), dtype=torch.float32)}
      edata_schemes={'label': Scheme(shape=(), dtype=torch.int64), 'h': Scheme(shape=(40,), dtype=torch.float32), 'train_mask': Scheme(shape=(), dtype=torch.bool)})

In [24]:
G_test = load_graphs("./mydata/unswnb15_test_data2.bin")[0][0]

In [25]:
G_test.ndata['h'] = th.ones(G_test.num_nodes(), G_test.edata['h'].shape[1]) 

In [26]:
Counter(G_test.edata['label'].numpy())

Counter({0: 299832,
         4: 129288,
         1: 26716,
         3: 9812,
         2: 8392,
         9: 1606,
         6: 14548,
         7: 104,
         8: 1398,
         5: 906})

In [26]:
Counter(G.edata['label'].cpu().numpy())

Counter({0: 699583,
         4: 301674,
         1: 62334,
         3: 22894,
         2: 19582,
         6: 33944,
         8: 3260,
         9: 3748,
         7: 244,
         5: 2116})

In [27]:
G

Graph(num_nodes=406258, num_edges=1149379,
      ndata_schemes={'h': Scheme(shape=(40,), dtype=torch.float32)}
      edata_schemes={'label': Scheme(shape=(), dtype=torch.int64), 'h': Scheme(shape=(40,), dtype=torch.float32), 'train_mask': Scheme(shape=(), dtype=torch.bool)})

In [28]:
G.edata['h'].shape

torch.Size([1149379, 40])

In [ ]:
class GATLayer(nn.Module):
    def __init__(self, in_dim , out_dim):
        super(GATLayer, self).__init__()
        self.W = nn.Linear(in_dim, out_dim, bias=False)
        self.attn_fc = nn.Linear(3 * out_dim, 1, bias=False)
        self.reset_parameters()
    def reset_parameters(self):
        gain = nn.init.calculate_gain('relu')
        nn.init.xavier_normal_(self.W.weight, gain=gain)
        nn.init.xavier_normal_(self.attn_fc.weight, gain=gain)
    def edge_attention(self, edges):
        z = torch.cat([edges.src['z'], edges.dst['z'], edges.data['z']], -1)
        a = self.attn_fc(z)
        alpha = F.leaky_relu(a)
        return {'e': alpha}
    def message_func(self, edges):
        return {'z': edges.src['z'], 'e': edges.data['e'] ,'m': edges.data['z']}
    def reduce_func(self, nodes):
        alpha = F.softmax(nodes.mailbox['e'], dim=1) # 归一化每一条入边的注意力系数
        h = torch.sum(alpha * nodes.mailbox['m'], dim=1)
        return {'h': F.relu(h)}
    def forward(self, g_dgl, hfeat, efeat):
        with g_dgl.local_scope():
            g = g_dgl
            z1 = self.W(hfeat)
            z2 = self.W(efeat)
            g.ndata['z'] = z1 # 每个节点的特征
            g.edata['z'] = z2 # 每条边的特征
            g.apply_edges(self.edge_attention) # 为每一条边获得其注意力系数
            g.update_all(self.message_func, self.reduce_func)
            return g.ndata['h'], g.edata['z']
class Model(nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim):
        super().__init__()
        self.layer1 = GATLayer(in_dim, hidden_dim)
        self.layer2 = GATLayer(hidden_dim, out_dim)
        self.pred = MLPPredictor(out_dim, 5)
    def forward(self, g, nfeats, efeats):
        nfeats, efeats = self.layer1(g, nfeats, efeats)
#         nfeats, efeats = self.layer2(g, nfeats, efeats)
        return self.pred(g, nfeats)
class MLPPredictor(nn.Module):
    def __init__(self, in_features, out_classes):
        super().__init__()
        self.W = nn.Linear(in_features * 2, out_classes)
    def apply_edges(self, edges):
        h_u = edges.src['h']
        h_v = edges.dst['h']
        score = self.W(th.cat([h_u, h_v], 1))
        return {'score': score}
    def forward(self, graph, h):
        with graph.local_scope():
            graph.ndata['h'] = h
            graph.apply_edges(self.apply_edges)
            return graph.edata['score']